In [2]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('testData.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

In [3]:
entities = interpreter.parse("AAPL")["intent"]

print(entities)

{'name': 'stock_search', 'confidence': 0.45701349589671014}


In [4]:
import spacy
nlp = spacy.load("en_core_web_md")



include_entities = ['ORG']
# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
   
    # Create a spacy document
    doc = nlp(message)

    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            return ent.text
    return None

In [5]:
print(extract_entities("IBM"))

IBM


In [26]:
from iexfinance.stocks import Stock

appl = Stock('AAPL', token="sk_5949b3f3377c4d278e80fa93394b9bda")

quote = appl.get_quote()
template = "{0} in {1} have latest price {2} updated in {3}"
template.format(template,quote["companyName"],quote['primaryExchange'],str(quote['latestPrice']),quote['latestTime'])


{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'NASDAQ',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'close': None,
 'closeTime': None,
 'high': None,
 'low': None,
 'latestPrice': 201.71,
 'latestSource': 'IEX real time price',
 'latestTime': '11:53:03 AM',
 'latestUpdate': 1565625183508,
 'latestVolume': None,
 'iexRealtimePrice': 201.71,
 'iexRealtimeSize': 3,
 'iexLastUpdated': 1565625183508,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 200.99,
 'previousVolume': None,
 'change': 0.72,
 'changePercent': 0.00358,
 'volume': None,
 'iexMarketPercent': 0.028589526589928502,
 'iexVolume': 292787,
 'avgTotalVolume': 25771974,
 'iexBidPrice': 201.71,
 'iexBidSize': 100,
 'iexAskPrice': 201.73,
 'iexAskSize': 100,
 'marketCap': 911563797800,
 'peRatio': 17.05,
 'week52High': 233.47,
 'week52Low': 142,
 'ytdChange': 0

In [6]:
import random
from iexfinance.stocks import Stock
INIT = 0
SEARCH_STOCK = 1
STOCK_NOT_FOUND = 2
DATA_OK = 3
DATA_FAILED = 4
BYE_OK = 5
BYE_FAILED = 6


# a dictionary with (state,intent) -> (state,respond messages)
policy = {
    (INIT,"greet"):(INIT,["Hi, I am your stock assistant, which stock do you want to search?"]),
    (INIT,"ask_explanation"):(INIT,["I am a stock assistant, you can ask any your favorite stock.","I can help you to check your stock, please tell me the stock number."]),
    (INIT,"stock_search"):(SEARCH_STOCK,["no problem. Wait me a second….. "]),
    (STOCK_NOT_FOUND,None):(STOCK_NOT_FOUND,["could you give me the stock number please?"]),
    (STOCK_NOT_FOUND,"stock_search"):(SEARCH_STOCK,["OK, your stock info is on the way."]),
    (DATA_OK,None):(DATA_OK,["This is your stock information : {0}"]),
    (DATA_OK,"stock_search"):(SEARCH_STOCK,["OK, another stock is on the way"]),
    (DATA_OK,"goodbye"):(BYE_OK,["It’s my pleasure to help you."]),
    (DATA_FAILED,None):(DATA_FAILED,["Sorry, I can't check that information, could you provide a valid number?"]),
    (DATA_FAILED,"stock_search"):(SEARCH_STOCK,["On my duty!"]),
    (DATA_FAILED,"goodbye"):(BYE_FAILED,[" I am sorry that can’t help you T_T."])
    ## add additional unmatched dict
    
}
# function that find stock information
def find_stock(stock_name):
    token ="sk_5949b3f3377c4d278e80fa93394b9bda"
    stock = Stock(stock_name,token=token)
    quote = stock.get_quote()
    template = "{} in {} have latest price {} updated in {}"
    
    return template.format(quote["companyName"],quote['primaryExchange'],str(quote['latestPrice']),quote['latestTime'])
# function that interpret the intent and entities
def interpret (message):
    intent = interpreter.parse(message)["intent"]["name"]
    entities = interpreter.parse(message)["entities"]
    return intent, entities
def execute_policy(state,intent,info = ""):
    next_state,responses =policy[(state,intent)]
    msg = None
    if state == DATA_OK and intent is None:
        msg = random.choice(responses).format(info)
        print("bot: "+random.choice(responses).format(info))
    else:
        msg = random.choice(responses)
        print("bot: "+random.choice(responses))
        
    return next_state, msg
def respond(state,message):
    intent,entities = interpret(message)
    new_state = None
    
    new_state,msg = execute_policy(state,intent)
   
    if new_state == SEARCH_STOCK:
        stock_value = None
        stock_value = extract_entities(message)
        if stock_value is  None: #len(entities) == 0:
            
            new_state,msg = execute_policy(STOCK_NOT_FOUND,None)
        else:
           # stock_value = entities[0]["value"]
            print(stock_value)
            stock_info = find_stock(stock_value)
            if stock_info is not None:
                new_state,msg = execute_policy(DATA_OK,None,info=stock_info)
            else:
                new_state,msg = execute_policy(DATA_FAILED,None)
    
    return new_state,msg

def respond_messages(messages):
    state = INIT
    for message in messages:
        print("user: "+message)
        state,msg = respond(state,message)
        print(msg)

messages = ["Hello","Could you help me to check some stock information?","TSLA","good bye"]

respond_messages(messages)
        
        
        

user: Hello
bot: Hi, I am your stock assistant, which stock do you want to search?
Hi, I am your stock assistant, which stock do you want to search?
user: Could you help me to check some stock information?
bot: no problem. Wait me a second….. 
bot: could you give me the stock number please?
could you give me the stock number please?
user: TSLA
bot: OK, your stock info is on the way.
TSLA
bot: This is your stock information : Tesla, Inc. in NASDAQ have latest price 232.72 updated in 11:29:04 AM
This is your stock information : Tesla, Inc. in NASDAQ have latest price 232.72 updated in 11:29:04 AM
user: good bye
bot: It’s my pleasure to help you.
It’s my pleasure to help you.


In [24]:
# start robot


state = INIT




In [14]:
from wxpy import *
bot = Bot()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as Leben


In [25]:
my_friend = bot.friends().search('alexanderio')[0]

@bot.register(my_friend)
def reply_my_friend(msg):
    global state
    state,msg = respond(state,msg.text)
    return msg


bot: Hi, I am your stock assistant, which stock do you want to search?
bot: Hi, I am your stock assistant, which stock do you want to search?
bot: Hi, I am your stock assistant, which stock do you want to search?
bot: no problem. Wait me a second….. 
bot: could you give me the stock number please?
bot: OK, your stock info is on the way.
AAPL
bot: This is your stock information : Apple, Inc. in NASDAQ have latest price 208.54 updated in 11:54:28 AM


In [22]:
my_friend.send('Hello my friend')

↪ alexanderio : Hello my friend (Text)